import libraries

In [33]:
import tmdbsimple as tmdb
import requests
import pandas as pd
import time

Loop through 50 pages to create a dataframe of 1000 movies

In [62]:
key = open('key.txt','r').read()
payload = "{}"

movie_df = pd.DataFrame() #create empty dataframe to enable 'while loop' below

page=1
while movie_df.shape[0] < 1000:
    url = "https://api.themoviedb.org/3/movie/top_rated?api_key={0}&language=en-US&page={1}".format(key, str(page))
    response = requests.request("GET", url, data=payload).json()
    if page == 1: #initialize dataframe on first loop
        movie_df = pd.DataFrame(response['results'])
    else:
        movie_df = movie_df.append(pd.DataFrame(response['results']))
    
    movie_df = movie_df[movie_df['original_language']=='en'] #remove non english movies
    time.sleep(0.25) #rate limit is 4 pages per second
    page+=1
    
movie_df.reset_index(inplace=True,drop=True) #reset index since we dropped non english rows

#drop irrelevant columns for this analysis
dropCols = ['adult','backdrop_path', 'original_language','original_title', 'poster_path','video']

movie_df.drop(dropCols,axis=1,inplace=True)

In [39]:
movie_df.to_csv('data\movie_df.csv',encoding="utf-8",index=False)
movie_df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg,"[18, 10749]",337167,en,Fifty Shades Freed,Believing they have left behind shadowy figure...,566.277884,/jjPJ4s3DWZZvI4vw8Xfi4Vqa1Q8.jpg,2018-02-07,Fifty Shades Freed,False,6.1,1126
1,False,/mhdeE1yShHTaDbJVdWyTlzFvNkr.jpg,"[16, 12, 10751, 35]",269149,en,Zootopia,"Determined to prove herself, Officer Judy Hopp...",341.104188,/sM33SANp9z6rXW8Itn7NnG1GOEs.jpg,2016-02-11,Zootopia,False,7.7,6697
2,False,/ut3n6iLMF79vDqyKFU6PeojZZpQ.jpg,"[28, 12, 14, 878]",284054,en,Black Panther,King T'Challa returns home from America to the...,248.787619,/uxzzxijgPIY7slzFvMotPv8wjKA.jpg,2018-02-13,Black Panther,False,7.4,3599
3,False,/c4Dw37VZjBmObmJw9bmt8IDwMZH.jpg,"[28, 14, 12, 878]",181808,en,Star Wars: The Last Jedi,Rey develops her newly discovered abilities wi...,229.525757,/kOVEVeg59E0wsnXmF9nrh6OmWII.jpg,2017-12-13,Star Wars: The Last Jedi,False,7.1,4583
4,False,/askg3SMvhqEl4OL52YuvdtY40Yb.jpg,"[12, 35, 10751, 16]",354912,en,Coco,Despite his family’s baffling generations-old ...,210.050883,/eKi8dIrr8voobbaGzDpe8w0PVbC.jpg,2017-10-27,Coco,False,7.8,3474


**Genres**

In [53]:
url = "https://api.themoviedb.org/3/genre/movie/list?api_key={0}&language=en-US&page={1}".format(key, str(page))
response = requests.request("GET", url, data=payload).json()

genreDict = dict(zip([i['id'] for i in response['genres']],
                     [i['name'] for i in response['genres']]))

genreDict

{12: 'Adventure',
 14: 'Fantasy',
 16: 'Animation',
 18: 'Drama',
 27: 'Horror',
 28: 'Action',
 35: 'Comedy',
 36: 'History',
 37: 'Western',
 53: 'Thriller',
 80: 'Crime',
 99: 'Documentary',
 878: 'Science Fiction',
 9648: 'Mystery',
 10402: 'Music',
 10749: 'Romance',
 10751: 'Family',
 10752: 'War',
 10770: 'TV Movie'}